Run each cell in order to create a dataset of 180000 inputs and outputs and save to drive

In [1]:
import numpy as np
from numpy import random
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
T = 48
Q = 8 # T/Q should be an integer
N = int(T/Q)
A = 440 # Hz
guitar = [[A*pow(2,(-29+i)/12) for i in range(24)],
          [A*pow(2,(-24+i)/12) for i in range(24)],
          [A*pow(2,(-19+i)/12) for i in range(24)],
          [A*pow(2,(-14+i)/12) for i in range(24)],
          [A*pow(2,(-10+i)/12) for i in range(24)],
          [A*pow(2,(-5+i)/12) for i in range(24)]]

def next_key(chords):
  next = [[[0 for i in range(3)] for j in range(6)] for k in range(6)]
  for i in range(6):
    for j in range(6):
      next[i][j] = chords[i][j].copy()
      next[i][j][2] = next[i][j][2] + 1
    if next[i][4][2] == 12:
      next[i].pop()
      next[i].insert(0,next[i][4].copy())
      next[i][0][2] = next[i][0][2] - 12
  return next

def chord2tab(chord,mute_prob=0.1):
  m = 0 # mute token
  tab = [m for i in range(6)]
  color = chord[0]
  shape = chord[1]
  fret = chord[2]

  if color == 0: # major chords
    if shape == 0: tab = [m,4,3,1,2,1]
    elif shape == 1: tab = [m,1,3,3,3,1]
    elif shape == 2: tab = [4,3,1,1,4,4]
    elif shape == 3: tab = [1,3,3,2,1,1]
    elif shape == 4: tab = [m,m,1,3,4,3]
  elif color == 1: # minor chords
    if shape == 0: tab = [m,4,2,1,2,m]
    elif shape == 1: tab = [m,1,3,3,2,1]
    elif shape == 2: tab = [4,2,1,1,4,4]
    elif shape == 3: tab = [1,3,3,1,1,1]
    elif shape == 4: tab = [m,m,1,3,4,2]

  tab = [tab[i] if random.rand() > mute_prob else m for i in range(6)]

  return [tab[i] + fret if tab[i] != m else m for i in range(6)]

def freq2bin(freq):
  bin_ceils = [(440*2**((i-29)/12) + 440*2**((i-28)/12))/2 for i in range(T)]
  for i in range(T):
    if freq < bin_ceils[i]: return i
  return T

def freqs2tones(freqs):
  # length T array of only 0s and 1s
  tones = [0 for i in range(T)]
  for i in range(len(freqs)):
    if freqs[i] != 0:
      bin = freq2bin(freqs[i])
      tones[bin] = 1

  # convert length T binary array to length N sequence of tokens. Tokens are made by converting sets of T/N binary entries in tones to decimal
  tokens = [0 for i in range(N)]
  binaries = [pow(2,i) for i in range(Q)]
  for i in range(N):
    b = tones[i*Q:i*Q+Q]
    tokens[i] = np.inner(binaries,b)

  return tokens

def tab2freqs(tab):
  freqs = [guitar[i][tab[i]-1] if tab[i] != 0 else 0 for i in range(6)]
  return freqs2tones(freqs)

def make_tones2tab_dataset(mute_prob=0.1):
  # specify csf chord representations
  c = [[[0,0,0],[0,1,3],[0,2,5],[0,3,8],[0,4,10],[0,0,12]], # 1
      [[1,4,0],[1,0,2],[1,1,5],[1,2,7],[1,3,10],[1,4,12]], # 2
      [[1,3,0],[1,4,2],[1,0,4],[1,1,7],[1,2,9],[1,3,12]], # 3
      [[0,3,1],[0,4,3],[0,0,5],[0,1,8],[0,2,10],[0,3,13]], # 4
      [[0,2,0],[0,3,3],[0,4,5],[0,0,7],[0,1,10],[0,2,12]], # 5
      [[1,1,0],[1,2,2],[1,3,5],[1,4,7],[1,0,9],[1,1,12]]] # 6
  csharp = next_key(c)
  d = next_key(csharp)
  dsharp = next_key(d)
  e = next_key(dsharp)
  f = next_key(e)
  fsharp = next_key(f)
  g = next_key(fsharp)
  gsharp = next_key(g)
  a = next_key(gsharp)
  asharp = next_key(a)
  b = next_key(asharp)
  keys = [c,csharp,d,dsharp,e,f,fsharp,g,gsharp,a,asharp,b]

  # store as tablature
  desired_dataset_length = 180000
  cycles = int(desired_dataset_length/(12*6*6))
  tabs = np.zeros((cycles,12,6,6,6))
  for l in range(cycles):
    for i in range(len(keys)):
      for j in range(6):
        #rand = random.randint(1,13)
        for k in range(6):
          #tabs[l][i][j][k] = [random.randint(rand,rand+3), random.randint(rand,rand+3), random.randint(rand,rand+3), random.randint(rand,rand+3), random.randint(rand,rand+3), random.randint(rand,rand+3)]
          tabs[l][i][j][k] = chord2tab(keys[i][j][k].copy())

  # reshape
  tabs = tf.cast(np.reshape(tabs, (tabs.shape[0]*tabs.shape[1]*tabs.shape[2]*tabs.shape[3],6)),tf.int32)

  # generate outputs after randomly muting strings
  tabs = [[tabs[i][j] if random.rand() > mute_prob else 0 for j in range(6)] for i in range(len(tabs))]
  tones = [tab2freqs(tabs[i]) for i in range(len(tabs))]

  # convert inputs and outputs to tensors
  inputs = tf.convert_to_tensor(tones, dtype=tf.int32)
  outputs = tf.convert_to_tensor(tabs, dtype=tf.int32)
  print()
  print('Shape of the input tensors: ' + str(tf.shape(inputs)))
  print('Shape of the output tensors: ' + str(tf.shape(outputs)))
  print()

  # show example
  idx = 8
  print('Input example: ' + str(inputs[idx].numpy()))
  print('Resulting output: ' + str(outputs[idx].numpy()))

  return inputs, outputs

In [4]:
inputs, outputs = make_tones2tab_dataset()


Shape of the input tensors: tf.Tensor([179712      6], shape=(2,), dtype=int32)
Shape of the output tensors: tf.Tensor([179712      6], shape=(2,), dtype=int32)

Input example: [ 0  4  0 34  0  0]
Resulting output: [0 6 0 0 7 6]


In [5]:
t2t_dataset_inputs = np.array(inputs)
np.save('/content/drive/My Drive/FretboardAI/t2t_dataset_inputs',t2t_dataset_inputs)

In [6]:
t2t_dataset_outputs = np.array(outputs)
np.save('/content/drive/My Drive/FretboardAI/t2t_dataset_outputs',t2t_dataset_outputs)